In [1]:
#DO NOT RUN!!! 
# needed to make it run on Alisa's laptop -- ignore otherwise 
import sys 
#sys.executable, sys.path
sys.path.append('/Users/alisa/workspace/Urdu_Fake_News/envname/lib/python3.7/site-packages')

In [12]:
from pathlib import Path
import numpy as np

## Reading Classification Results

In [2]:
%ls ./results/EMNLP/

y_probabilities_for_roccurve_MT.json
y_probabilities_for_roccurve_MT_Orignal
y_probabilities_for_roccurve_Mixed.json
y_probabilities_for_roccurve_Mixed_Downsize.json
y_probabilities_for_roccurve_Orignal_MT.json
y_probabilities_for_roccurve_Urdu.json


In [33]:
import json 

y_prob_Urdu = json.load(open(Path('./results/EMNLP/y_probabilities_for_roccurve_Urdu.json')))
y_prob_MT = json.load(open(Path('./results/EMNLP/y_probabilities_for_roccurve_MT.json')))
y_prob_Mixed = json.load(open(Path('./results/EMNLP/y_probabilities_for_roccurve_Mixed.json')))
y_prob_Mixed_Donwsized = json.load(open(Path('./results/EMNLP/y_probabilities_for_roccurve_Mixed_Downsize.json')))
y_prob_Orignal_MT = json.load(open(Path('./results/EMNLP/y_probabilities_for_roccurve_Orignal_MT.json')))
y_prob_MT_Orignal = json.load(open(Path('./results/EMNLP/y_probabilities_for_roccurve_MT_Orignal')))

y_prob_MT.keys()

dict_keys(['test_y', '2c-2w-2f', '2c-1w-0f', '1c-1w-0f', '2c-0w-0f', '0c-2w-0f'])

In [61]:
EXP_KEYS = ['2c-2w-2f', '2c-1w-0f', '1c-1w-0f', '2c-0w-0f', '0c-2w-0f']

In [46]:
# setting paths variables
CORPORA_DIR_PATH = Path('Corpora/')
TEST_REAL_PART_PATH = 'Test/Real'
TEST_FAKE_PART_PATH = 'Test/Fake'

RESULT_PATH = Path('results/EMNLP/Misclassification_Analysis/')
FAKE_AS_LEGIT_STR = 'fake_as_legit'
LEGIT_AS_FAKE_STR = 'legit_as_fake'

In [41]:
%ls results/EMNLP/Misclassification_Analysis/

1.Corpus/                3.Corpus_Mixed/          5.Cross_MT_Orignal/
2.CorpusMT/              4.Corpus_Mixed_Downsize/ 6.Cross_Orignal_MT/


## Misclassification Analysis

In [69]:
import numpy as np

def detect_real_fake_part(y_prob):
    return np.where((np.array(y_prob['test_y']) == 0) == False)[0][0]

In [81]:
import shutil 
import numpy as np
from pathlib import Path

def find_and_copy_missclassified_files(y_proba, corpus_name, threshold=0.5):    
    part_len = detect_real_fake_part(y_proba)    
        
    test_real_path =  Path(CORPORA_DIR_PATH, corpus_name,  TEST_REAL_PART_PATH)
    test_real_files_sorted = sorted(test_real_path.glob('*.txt'))
    
    test_fake_path =  Path(CORPORA_DIR_PATH, corpus_name,  TEST_FAKE_PART_PATH)
    test_fake_files_sorted = sorted(test_fake_path.glob('*.txt'))
    
    total_legit_as_fake = 0 
    total_fake_as_legit = 0 
    
    for exp in EXP_KEYS:
        # legit as fake 
        misclassified_legit_as_fake_idxs = np.where((np.array(y_proba[exp][:part_len]) >= threshold) == True)[0]
        total_legit_as_fake += len(misclassified_legit_as_fake_idxs)
        legit_as_fake_dir = Path(RESULT_PATH, corpus_name, exp, LEGIT_AS_FAKE_STR)        
        for file_idx in misclassified_legit_as_fake_idxs: 
            src_file = test_real_files_sorted[file_idx]
            dest_file = Path(legit_as_fake_dir, src_file.name)
            shutil.copyfile(src_file, dest_file)
                        
        # fake as legit     
        misclassified_fake_as_legit_idxs = np.where((np.array(y_proba[exp][part_len:]) >= threshold) == False)[0]    
        total_fake_as_legit += len(misclassified_fake_as_legit_idxs)
        fake_as_legit_dir = Path(RESULT_PATH,  corpus_name, exp, FAKE_AS_LEGIT_STR)
        for file_idx in misclassified_fake_as_legit_idxs: 
            src_file = test_fake_files_sorted[file_idx]
            dest_file = Path(fake_as_legit_dir, src_file.name)
            shutil.copyfile(src_file, dest_file)
            
    return total_legit_as_fake, total_fake_as_legit          

In [75]:
corpora = ['1.Corpus', '2.CorpusMT', '3.Corpus_Mixed', '4.Corpus_Mixed_Downsize', '5.Cross_MT_Orignal', '6.Cross_Orignal_MT']

In [83]:
y_probabs = {}
y_probabs[corpora[0]] = y_prob_Urdu
y_probabs[corpora[1]] = y_prob_MT
y_probabs[corpora[2]] = y_prob_Mixed
y_probabs[corpora[3]] = y_prob_Mixed_Donwsized
y_probabs[corpora[4]] = y_prob_MT_Orignal
y_probabs[corpora[5]] = y_prob_Orignal_MT

In [84]:
for corpus in corpora: 
    print(corpus)
    find_and_copy_missclassified_files(y_probabs[corpus], corpus)

1.Corpus
1.Corpus 150
2.CorpusMT
2.CorpusMT 60
3.Corpus_Mixed
3.Corpus_Mixed 210
4.Corpus_Mixed_Downsize
4.Corpus_Mixed_Downsize 147
5.Cross_MT_Orignal
5.Cross_MT_Orignal 150
6.Cross_Orignal_MT
6.Cross_Orignal_MT 60
